In [1]:
import pandas as pd
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true") 
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/03 15:39:28 WARN Utils: Your hostname, DESKTOP-9VIST87 resolves to a loopback address: 127.0.1.1; using 172.28.235.189 instead (on interface eth0)
22/10/03 15:39:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 15:39:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Adding postcode income as feature 

This notebook finds each merchant's main consumer postcodes and adds that postcodes' median weekly personal income as a feature of the merchat for further ranking and testing.

First reduce some of the transactions, as the income dataframe have some missing postcodes.

In [15]:
# Read the transaction dataframe
transaction_df = pd.read_parquet('../data/curated/transactions_detail.parquet')

In [16]:
transaction_df

,user_id,dollar_value,order_id,merchant_name,merchant_abn,take_rate,type,consumer_address,state,postcode,gender
0,19,7.399023,57be0f0a-2e77-4f97-a975-ed03ab471739,Et PC,75944642726,6.03,tent and awning shops,68657 Johnson Glen Suite 266,TAS,7276,Female
1,19,5.589068,0dce20ed-460d-43b4-9dad-208e8d6ca555,Urna Incorporated,78905182348,4.65,"watch, clock, and jewelry repair shops",68657 Johnson Glen Suite 266,TAS,7276,Female
2,19,80.534344,9253c118-5861-4a0f-baab-91db1de0079e,Ut Eros Institute,39088614710,5.67,"opticians, optical goods, and eyeglasses",68657 Johnson Glen Suite 266,TAS,7276,Female
3,19,24.437234,5f5e74a8-6b0e-4de2-ac48-7c75a1db789e,Sodales Incorporated,29936037423,1.97,"hobby, toy and game shops",68657 Johnson Glen Suite 266,TAS,7276,Female
4,19,55.852933,8d2f86b0-f421-4b3a-b822-18105f0140fb,Sem LLC,91873181182,6.00,"digital goods: books, movies, music",68657 Johnson Glen Suite 266,TAS,7276,Female
...,...,...,...,...,...,...,...,...,...,...,...
12047312,24024,19.032062,aa3c2d60-4152-4eb6-93bb-d642b20545dd,Vehicula Pellentesque Corporation,63290521567,6.48,artist supply and craft shops,064 Steven Mountain Suite 468,VIC,3966,Female
12047313,24024,151.562467,200b4391-e22d-4578-b54e-4df97aef0c5d,Lacus Vestibulum Institute,41383736952,2.25,"cable, satellite, and other pay television and...",064 Steven Mountain Suite 468,VIC,3966,Female
12047314,24024,44.334337,8567da33-47bb-4543-92f5-c5a0e126b736,Diam Eu Dolor PC,39211701585,4.76,shoe shops,064 Steven Mountain Suite 468,VIC,3966,Female
12047315,24024,22.764600,b60cb7c8-e87f-4870-81d0-6b65533cdded,Lacus Consulting,45629217853,6.98,"gift, card, novelty, and souvenir shops",064 Steven Mountain Suite 468,VIC,3966,Female


In [ ]:
# Read the income dataframe
income_df = pd.read_csv("../data/income/2021_income.csv")
income_df['POA_CODE_2021'] = income_df['POA_CODE_2021'].astype(int)

In [17]:
# Merge the income and transaction dataframe
transaction_df['postcode'] = transaction_df['postcode'].astype(int)
merged_df = transaction_df \
    .merge(income_df[['POA_CODE_2021']], left_on='postcode', right_on='POA_CODE_2021') \
    .drop('POA_CODE_2021', axis=1)
merged_df

,user_id,dollar_value,order_id,merchant_name,merchant_abn,take_rate,type,consumer_address,state,postcode,gender
0,19,7.399023,57be0f0a-2e77-4f97-a975-ed03ab471739,Et PC,75944642726,6.03,tent and awning shops,68657 Johnson Glen Suite 266,TAS,7276,Female
1,19,5.589068,0dce20ed-460d-43b4-9dad-208e8d6ca555,Urna Incorporated,78905182348,4.65,"watch, clock, and jewelry repair shops",68657 Johnson Glen Suite 266,TAS,7276,Female
2,19,80.534344,9253c118-5861-4a0f-baab-91db1de0079e,Ut Eros Institute,39088614710,5.67,"opticians, optical goods, and eyeglasses",68657 Johnson Glen Suite 266,TAS,7276,Female
3,19,24.437234,5f5e74a8-6b0e-4de2-ac48-7c75a1db789e,Sodales Incorporated,29936037423,1.97,"hobby, toy and game shops",68657 Johnson Glen Suite 266,TAS,7276,Female
4,19,55.852933,8d2f86b0-f421-4b3a-b822-18105f0140fb,Sem LLC,91873181182,6.00,"digital goods: books, movies, music",68657 Johnson Glen Suite 266,TAS,7276,Female
...,...,...,...,...,...,...,...,...,...,...,...
10061113,9579,95.722241,7a0ae3a0-14f1-4dc6-8117-4f18ef674274,Lobortis Ultrices Company,64403598239,6.31,"music shops - musical instruments, pianos, and...",6054 Kelly Cove Apt. 926,WA,6208,Male
10061114,9579,161.360897,bfe300cb-49b5-460c-aca4-8c9cede4d54e,Cubilia Curae Donec Inc.,32176774610,5.10,"watch, clock, and jewelry repair shops",6054 Kelly Cove Apt. 926,WA,6208,Male
10061115,9579,10.956257,208b34a2-cc7c-43a3-83ca-8be6e1a0bda0,Placerat Eget Venenatis Limited,68216911708,3.05,"computers, computer peripheral equipment, and ...",6054 Kelly Cove Apt. 926,WA,6208,Male
10061116,9579,16.354819,e8fd0344-1a5a-4130-ad05-380b6ab56c87,Lorem Foundation,21025433654,6.55,"digital goods: books, movies, music",6054 Kelly Cove Apt. 926,WA,6208,Male


In [18]:
merged_df.to_parquet(r'../data/curated/reduced_transactions.parquet')

In [4]:
transaction_df = spark.read.parquet('../data/curated/reduced_transactions.parquet')
transaction_df

user_id,dollar_value,order_id,merchant_name,merchant_abn,take_rate,type,consumer_address,state,postcode,gender,__index_level_0__
19,7.399022756549412,57be0f0a-2e77-4f9...,Et PC,75944642726,6.03,tent and awning s...,68657 Johnson Gle...,TAS,7276,Female,0
19,5.589067937936061,0dce20ed-460d-43b...,Urna Incorporated,78905182348,4.65,"watch, clock, and...",68657 Johnson Gle...,TAS,7276,Female,1
19,80.5343441925844,9253c118-5861-4a0...,Ut Eros Institute,39088614710,5.67,"opticians, optica...",68657 Johnson Gle...,TAS,7276,Female,2
19,24.437233660861473,5f5e74a8-6b0e-4de...,Sodales Incorporated,29936037423,1.97,"hobby, toy and ga...",68657 Johnson Gle...,TAS,7276,Female,3
19,55.85293310708887,8d2f86b0-f421-4b3...,Sem LLC,91873181182,6.0,digital goods: bo...,68657 Johnson Gle...,TAS,7276,Female,4
19,448.73714025037685,43bfa43f-9f00-446...,Elit Aliquam Auct...,39759375662,6.96,"books, periodical...",68657 Johnson Gle...,TAS,7276,Female,5
19,17.373094025584514,5372cab3-a266-4d6...,Non Vestibulum In...,49891706470,5.8,tent and awning s...,68657 Johnson Gle...,TAS,7276,Female,6
19,21.696374423491548,7f41e812-d90a-492...,Dui Nec Corporation,19933438190,2.22,tent and awning s...,68657 Johnson Gle...,TAS,7276,Female,7
19,2809.102512823842,0470fa28-6af7-4a6...,Venenatis Lacus E...,73918814753,4.99,"furniture, home f...",68657 Johnson Gle...,TAS,7276,Female,8
19,4.75460797755676,4a767683-7da0-43c...,Suspendisse Dui C...,46804135891,2.93,"opticians, optica...",68657 Johnson Gle...,TAS,7276,Female,9


Now count and see which postcode of each merchant has the highest number of transactions.

In [5]:
transaction_count_df = transaction_df.groupBy('merchant_name', 'postcode').count()
transaction_count_df = transaction_count_df.withColumnRenamed("count","transaction_count")

In [6]:
from pyspark.sql import Window
import pyspark.sql.functions as f

w = Window.partitionBy('merchant_name')
max_count_df = transaction_count_df.withColumn('max_count', f.max('transaction_count').over(w))\
                .where(f.col('transaction_count') == f.col('max_count'))\
                .drop('max_count')\
                .dropDuplicates(['merchant_name'])


Then include the postcode's income into the merchant detail, as a new feature.

In [3]:
merchant_detail = spark.read.parquet('../data/curated/merchant_detail.parquet')

In [7]:
merchant_detail = merchant_detail.join(max_count_df.drop(col("transaction_count")),['merchant_name'],how='inner')

In [8]:
merchant_detail = merchant_detail.toPandas()
merchant_detail['postcode'] = merchant_detail['postcode'].astype(int)

In [10]:
merged_df = merchant_detail \
    .merge(income_df[['POA_CODE_2021', 'Median_tot_prsnl_inc_weekly']], left_on='postcode', right_on='POA_CODE_2021') \
    .drop('POA_CODE_2021', axis=1)


In [11]:
merged_df

,merchant_name,merchant_abn,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit,postcode,Median_tot_prsnl_inc_weekly
0,A Associates,14955471775,"opticians, optical goods, and eyeglasses",4.95,24797.58,71,349.0,68.0,3896,708
1,A Feugiat Inc.,20352820919,art dealers and galleries,2.51,24456.86,19,1287.0,34.0,3896,708
2,Elit Elit PC,96809815920,health and beauty spas,5.97,15284.20,35,437.0,51.0,3896,708
3,A Neque LLP,79081516258,"antique shops - sales, repairs, and restoratio...",3.45,58451.53,5,11690.0,112.0,2334,803
4,Imperdiet Non Limited,36800144612,"music shops - musical instruments, pianos, and...",6.84,67325.92,138,488.0,256.0,2334,803
...,...,...,...,...,...,...,...,...,...,...
4019,Sit Amet Ltd,13041890826,"music shops - musical instruments, pianos, and...",4.33,113286.19,1543,73.0,273.0,3926,886
4020,Sollicitudin Commodo Ipsum Industries,10714068705,"furniture, home furnishings and equipment shop...",2.51,388618.18,3064,127.0,542.0,2311,627
4021,Tempus Institute,63549078265,artist supply and craft shops,6.01,118083.82,120,984.0,394.0,5701,412
4022,Ultrices Industries,69431337574,telecom,1.64,34215.10,8,4277.0,31.0,3300,695


In [12]:
merged_df.to_csv(r'../data/curated/merchant_detail_with_income.csv', index = False, header=True)